In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# ssl._create_default_https_context = ssl._create_unverified_context

tf.set_random_seed(1)
np.random.seed(1)

BATCH_SIZE = 50
LR = 0.001              # learning rate

imageBigDataORL = np.zeros((400, 112,92))
YBigData = np.zeros((400,1))

#read_in

for people in range(1,41):
    for face in range(1,11):
        path = './ORL/s%d_%d.bmp' % (people, face)
        oriImage = PIL.Image.open(path)
        imageArray = np.array(oriImage)
#             print(imageArray)
        imageVec = np.reshape(imageArray, imageArray.shape[0] * imageArray.shape[1])
        # print(imageArray.shape)
        # print(imageBigDataORL.shape)
#             print((people - 1) * 10 + face)
        imageBigDataORL[(people - 1) * 10 + face - 1] = imageArray
        YBigData[(people - 1) * 10 + face - 1] = people

# onehot and other pre-process


# print(imageBigDataORL)
# print(YBigData)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(YBigData)
Big_Y = enc.transform(YBigData).toarray()
print(Big_Y)

# mnist = input_data.read_data_sets('./mnist', one_hot=True)  # they has been normalized to range (0,1)
# test_x = mnist.test.images[:2000]
# test_y = mnist.test.labels[:2000]

# # plot one example
# print(mnist.train.images.shape)     # (55000, 28 * 28)
# print(mnist.train.labels.shape)   # (55000, 10)
# plt.imshow(mnist.train.images[0].reshape((28, 28)), cmap='gray')
# plt.title('%i' % np.argmax(mnist.train.labels[0])); plt.show()



[[[ 39.  44.  53. ...  47.  48.  47.]
  [ 44.  44.  61. ...  47.  45.  46.]
  [ 59.  39.  57. ...  46.  46.  44.]
  ...
  [ 49.  46.  42. ...  30.  32.  28.]
  [ 46.  35.  34. ...  30.  29.  27.]
  [ 32.  37.  27. ...  29.  26.  29.]]

 [[ 34.  34.  33. ...   1.   1.   1.]
  [ 35.  30.  32. ...  38.  47.  39.]
  [ 34.  28.  31. ...  55.  59.  49.]
  ...
  [ 30.  29.  31. ...  35.  37.  41.]
  [ 31.  29.  32. ...  42.  34.  39.]
  [ 28.  33.  27. ...  37.  40.  33.]]

 [[ 39.  44.  53. ...  47.  48.  47.]
  [ 44.  44.  61. ...  47.  45.  46.]
  [ 59.  39.  57. ...  46.  46.  44.]
  ...
  [ 49.  46.  42. ...  30.  32.  28.]
  [ 46.  35.  34. ...  30.  29.  27.]
  [ 32.  37.  27. ...  29.  26.  29.]]

 ...

 [[125. 119. 124. ... 122. 120. 116.]
  [121. 124. 118. ... 119. 120. 120.]
  [122. 125. 122. ... 121. 118. 121.]
  ...
  [ 85.  93.  82. ...  48.  35.  43.]
  [ 87.  92.  85. ...  40.  39.  35.]
  [ 79.  93.  89. ...  36.  39.  40.]]

 [[119. 120. 120. ... 129. 128. 124.]
  [118. 121.

In [ ]:
tf_x = tf.placeholder(tf.float32, [None, 28*28]) / 255.
image = tf.reshape(tf_x, [-1, 112, 92, 1])              # (batch, height, width, channel)
tf_y = tf.placeholder(tf.int32, [None, 40])            # input y

# CNN
conv1 = tf.layers.conv2d(   # shape (28, 28, 1)
    inputs=image,
    filters=16,
    kernel_size=5,
    strides=1,
    padding='same',
    activation=tf.nn.relu
)           # -> (28, 28, 16)
pool1 = tf.layers.max_pooling2d(
    conv1,
    pool_size=2,
    strides=2,
)           # -> (14, 14, 16)
conv2 = tf.layers.conv2d(pool1, 32, 5, 1, 'same', activation=tf.nn.relu)    # -> (14, 14, 32)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2)    # -> (7, 7, 32)
flat = tf.reshape(pool2, [-1, 7*7*32])          # -> (7*7*32, )
output = tf.layers.dense(flat, 10)              # output layer

loss = tf.losses.softmax_cross_entropy(onehot_labels=tf_y, logits=output)           # compute cost
train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.metrics.accuracy(          # return (acc, update_op), and create 2 local variables
    labels=tf.argmax(tf_y, axis=1), predictions=tf.argmax(output, axis=1),)[1]

sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) # the local var is for accuracy_op
sess.run(init_op)     # initialize var in graph

# following function (plot_with_labels) is for visualization, can be ignored if not interested
from matplotlib import cm
try: from sklearn.manifold import TSNE; HAS_SK = True
except: HAS_SK = False; print('\nPlease install sklearn for layer visualization\n')
def plot_with_labels(lowDWeights, labels):
    plt.cla(); X, Y = lowDWeights[:, 0], lowDWeights[:, 1]
    for x, y, s in zip(X, Y, labels):
        c = cm.rainbow(int(255 * s / 9)); plt.text(x, y, s, backgroundcolor=c, fontsize=9)
    plt.xlim(X.min(), X.max()); plt.ylim(Y.min(), Y.max()); plt.title('Visualize last layer'); plt.show(); plt.pause(0.01)

plt.ion()
for step in range(600):
    b_x, b_y = mnist.train.next_batch(BATCH_SIZE)
    _, loss_ = sess.run([train_op, loss], {tf_x: b_x, tf_y: b_y})
    if step % 50 == 0:
        accuracy_, flat_representation = sess.run([accuracy, flat], {tf_x: test_x, tf_y: test_y})
        print('Step:', step, '| train loss: %.4f' % loss_, '| test accuracy: %.2f' % accuracy_)

        if HAS_SK:
            # Visualization of trained flatten layer (T-SNE)
            tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000); plot_only = 500
            low_dim_embs = tsne.fit_transform(flat_representation[:plot_only, :])
            labels = np.argmax(test_y, axis=1)[:plot_only]; plot_with_labels(low_dim_embs, labels)
plt.ioff()

# print 10 predictions from test data
test_output = sess.run(output, {tf_x: test_x[:10]})
pred_y = np.argmax(test_output, 1)
print(pred_y, 'prediction number')
print(np.argmax(test_y[:10], 1), 'real number')